# モデルファイル
最終的にはpyファイルにして、引数でデータのバージョン管理ができるようにすること

In [1]:
import os, sys, pdb
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import gc
%matplotlib inline
%load_ext autoreload
%autoreload 1
DATA_DIR = Path('../input')
MODEL_DIR = Path('../model')
OUT_DIR = Path('../output')
import warnings
warnings.filterwarnings('ignore')

In [2]:
from __future__ import print_function, division
import torch
from torch import nn
import torchvision
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
from random import shuffle
import random
random.seed(seed)
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms, utils
plt.ion()   # interactive mode

In [5]:
supported_rnns = {
    'lstm': nn.LSTM,
    'rnn': nn.RNN,
    'gru': nn.GRU
}
supported_rnns_inv = dict((v, k) for k, v in supported_rnns.items())

In [2]:
from eeglibrary import EEG, EEGDataSet, EEGDataLoader, EEGParser


In [3]:
class Args:
    def __init__(self):
        self.lr = 3e-4
        self.epochs = 50
        self.continue_from = False
        self.momentum = 0.9

In [ ]:
datasets = {
    part: EEGDataSet(data_dir / 'manifests' / '{}_{}_manifest.csv'.format(data_version, part), eeg_conf)
    for part in partial_name_list
}

data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataloaders = {part: EEGDataLoader(datasets[part], batch_size=2, num_workers=2) 
               for part in partial_name_list}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'
device

In [ ]:
# train.py
# if __name__ == '__main__':
# parse args
args = Args()
# set seeds: torch, cuda, np, random
# device setting
# make best model save dir
model_dir = model_dir
model_dir.mkdir(exist_ok=True)

roc_results, prec_results, rec_results = torch.Tensor(args.epochs), torch.Tensor(args.epochs), torch.Tensor(
    args.epochs)

best_roc = None

avg_loss, avg_auc, start_epoch, start_iter, optim_state = 0, 0, 0, 0, None

if args.continue_from:  # TODO: Starting from previous model
    raise NotImplementedError
    
model = models.vgg11_bn(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(class_names))
input_size = 224
model = model.to(device)

parameters = model.parameters()
optimizer = torch.optim.SGD(parameters, lr=args.lr,
                            momentum=args.momentum, nesterov=True, weight_decay=1e-5)

# if optim_state is not None:
#     optimizer.load_state_dict(optim_state)

# print(model)

criterion = nn.BCELoss()
batch_time = AverageMeter()
data_time = AverageMeter()
losses = AverageMeter()

for epoch in range(start_epoch, args.epochs):
    end = time.time()
    start_epoch_time = time.time()
    
    for phase in ['train', 'val']:
#         if phase == 'train':
#             scheduler.step()
        
        for i, inputs, labels in enumerate(dataloaders[phase]):
            # measure data loading time
            data_time.update(time.time() - end)
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                pdb.set_trace()
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
            avg_loss += loss.item()
            avg_auc += metrics.auc(labels, outputs)
            losses.update(loss_value, inputs.size(0))
            
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
            if True: #not args.silent:
                print('Epoch: [{0}][{1}/{2}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                    (epoch + 1), (i + 1), len(train_sampler), batch_time=batch_time, data_time=data_time, loss=losses))
            break
        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        